# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)


/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list
file_counter =0             #to count the number of files in the directory
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        file_counter+=1
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)
# The total number of found rows & files
print("The total amount of found rows are ",len(full_data_rows_list))
print(f"the total found file are {file_counter} files")
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist',
                     'firstName',
                     'gender',
                     'itemInSession',
                     'lastName',
                     'length',
                     'level',
                     'location',
                     'sessionId',
                     'song',
                     'userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0],
                         row[2],
                         row[3],
                         row[4],
                         row[5],
                         row[6],
                         row[7],
                         row[8],
                         row[12],
                         row[13],
                         row[16]))

The total amount of found rows are  8056
the total found file are 30 files


In [4]:
#print(full_data_rows_list)

In [5]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

# The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_keyspace')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
# here i will choose my primary key to be session_id ,iteminsession 
# but i want to check for its duplcation
df = pd.read_csv('event_datafile_new.csv')[['itemInSession','sessionId']]
df['itemInSession'] = df['itemInSession'].apply(str)
df['sessionId'] = df['sessionId'].apply(str)
df['check for duplication']=df['itemInSession']+','+df['sessionId']
df['check for duplication'].duplicated().any()
#if false we can make sure that from the source of data we never have the same 
#iteminsession with sessionid for more than one time  

False

In [10]:
# creating the table to answer question 1
# the query has a pertition key sessionId and a clustering column itemInSession and they were choosen to:
# - optimize the select statement (low latency) and was set after the where condition
# - prevent any duplication so as not to lose any row of source data because apache 
#   cassandra would overwrite a duplicated row
song_log_create_query = """
CREATE TABLE IF NOT EXISTS song_log 
(
session_id int,
iteminsession int,
artist text,
song_title text,
song_length float,
PRIMARY KEY (session_id,iteminsession)
)
"""
try:
    session.execute(song_log_create_query)
except Exception as e :
    print(e)


In [11]:
# here is to insert the data into specified columns
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# the insert query to insert data in columns        
        query = """INSERT INTO song_log (
                                        session_id,
                                        iteminsession,
                                        artist,
                                        song_title,
                                        song_length
                                        )"""
        query = query + "VALUES (%s,%s,%s,%s,%s)"
# choosing which value from the file 'event_datafile_new.csv' would be assigned for each column of our dataset
        session.execute(query, (
                                int(line[8]),
                                int(line[3]),
                                line[0],
                                line[9],
                                float(line[5])
                                ))


#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
# The SELECT statement to verify the data was entered into the table
query = """
    SELECT artist , song_title , song_length 
    from song_log
    where session_id = 338 and
    iteminsession = 4"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:         # to print the selected output
    print(row.artist , row.song_title , row.song_length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [13]:
# here i will choose my primary key to be user_id,session_id 
# but i want to check for its duplcation
df = pd.read_csv('event_datafile_new.csv')[['userId','sessionId']]
df['userId'] = df['userId'].apply(str)
df['sessionId'] = df['sessionId'].apply(str)
df['check for duplication']=df['userId']+','+df['sessionId']
df['check for duplication'].duplicated().any()
#if false we can make sure that from the source of data we never have the same 
#iteminsession with sessionid for more than one time  
# if true we have to use one more partition column so that to get the total source data without overwriting

True

In [14]:
# here i will choose my primary key to be user_id,itemInSession
df = pd.read_csv('event_datafile_new.csv')[['userId','itemInSession']]
df['userId'] = df['userId'].apply(str)
df['itemInSession'] = df['itemInSession'].apply(str)
df['check for duplication']=df['userId']+','+df['itemInSession']
df['check for duplication'].duplicated().any()

True

In [15]:
# here i will choose my primary key to be user_id,session_id,iteminsession
# but i want to check for its duplcation
df = pd.read_csv('event_datafile_new.csv')[['userId','sessionId','itemInSession']]
df['userId'] = df['userId'].apply(str)
df['sessionId'] = df['sessionId'].apply(str)
df['itemInSession'] = df['itemInSession'].apply(str)
df['check for duplication']=df['userId']+','+df['sessionId']+','+df['itemInSession']
df['check for duplication'].duplicated().any()
#if false we can make sure that from the source of data we never have the same 
#iteminsession with sessionid for more than one time  
# if true we have to use one more partition column so that to get the total source data without overwriting

False

In [16]:
# creating the table to answer question 2
# the query has a pertition key userId and a clustering column sessionId and itemInSession and they were choosen to:
# - optimize the select statement (low latency) and was set after the where condition
# - prevent any duplication so as not to lose any row of source data because apache 
#   cassandra would overwrite a duplicated row
artist_song_user_create_query = """
CREATE TABLE IF NOT EXISTS artist_song_user 
(
userId int,
sessionId int,
itemInSession int,
artist text,
song_title text,
user_first_name text,
user_last_name text,
PRIMARY KEY ((userId,sessionId),itemInSession)
)
"""
try:
    session.execute(artist_song_user_create_query)
except Exception as e :
    print(e)

In [17]:
# here is to insert the data into specified columns

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO artist_song_user (userId,
                                                sessionId,
                                                itemInSession,
                                                artist,
                                                song_title,
                                                user_first_name,
                                                user_last_name
                                                )"""
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
# choosing which value from the file 'event_datafile_new.csv' would be assigned for each column of our dataset        
        try:
            session.execute(query, (
                                    int(line[10]),
                                    int(line[8]),
                                    int(line[3]),
                                    line[0],
                                    line[9],
                                    line[1],
                                    line[4]
                                    ))
        except Exception as e:
            print(e)


In [18]:
# select statement to verify that data was entered correctly
query = """
    SELECT artist , song_title , user_first_name , user_last_name
    from artist_song_user
    where userId = 10 and
    sessionId = 182
    """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:               # to print selected columns
    print(row.artist ,",", row.song_title ,",", row.user_first_name,",",row.user_last_name)
 

Down To The Bone , Keep On Keepin' On , Sylvie , Cruz
Three Drives , Greece 2000 , Sylvie , Cruz
Sebastien Tellier , Kilometer , Sylvie , Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie , Cruz


In [19]:
# i will have to use song ,sessionid and iteminsession(as a primary key) to check for duplication
df = pd.read_csv('event_datafile_new.csv')[['song','userId']]
df['userId']=df['userId'].apply(str)
df['check for duplication']=df['song']+df['userId']
df['check for duplication'].duplicated().any()
#df.head()
#if false we can make sure that from the source of data we never have the same 
#iteminsession with sessionid for more than one time  
# if true we have to use one more partition column so that to get the total source data without overwriting

True

In [20]:
# Although there is duplicates but here we just only want to get which user listened to a certain song. It doesn't matter how many times or when.
# so i will keep using of only song and userId as the primary key 

In [21]:
# creating the table to answer question 3
# the query has a pertition key song and a clustering column sessionId and itemInSession and they were choosen to:
# - optimize the select statement (low latency) and was set after the where condition
# - prevent any duplication so as not to lose any row of source data because apache 
#   cassandra would overwrite a duplicated row

user_song_create_query = """
CREATE TABLE IF NOT EXISTS user_song 
(
song text,
userId int,
user_first_name text,
user_last_name text,
PRIMARY KEY (song,userId)  
)
"""
try:
    session.execute(user_song_create_query)
except Exception as e :
    print(e)
                    

In [22]:
# here is to insert the data into specified columns

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = """INSERT INTO user_song 
                                        (
                                        song,
                                        userId,
                                        user_first_name,
                                        user_last_name
                                         )"""
        query = query + "VALUES (%s,%s,%s,%s)"
# choosing which value from the file 'event_datafile_new.csv' would be assigned for each column of our dataset      
        session.execute(query,(
                               line[9],
                               int(line[10]),
                               line[1],
                               line[4]
                                ))


In [23]:
# The SELECT statement to verify the data was loaded into our dataset
query = """
    SELECT user_first_name , user_last_name
    from user_song
    where song = 'All Hands Against His Own'
    """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:               # print the selected columns
    print(row.user_first_name,",",row.user_last_name)
 

Jacqueline , Lynch
Tegan , Levine
Sara , Johnson


### Drop the tables before closing out the sessions

In [24]:
query = ["drop table IF EXISTS song_log","drop table IF EXISTS artist_song_user","drop table IF EXISTS user_song"]
try:
    for i in range(3):
        session.execute(query[i])
except Exception as e:
    print(e)


### Closing the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()